In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model 
from sklearn.metrics import confusion_matrix,r2_score,roc_curve,auc,precision_recall_curve,average_precision_score,mean_squared_error

In [2]:
songs = pd.read_csv('data_updated.csv')

In [3]:
del songs['Unnamed: 0']
del songs['id']
del songs['release_date']
del songs['artists']
del songs['name']
del songs['emotion']
songs['language'] = np.where(songs['language']=='English',1,0)
songs['popularity'] = np.where(songs['popularity']>=33, 1, 0)
songs['duration_ms'] = songs['duration_ms']/60000
songs.rename(columns={'duration_ms':'duration'})

,year,duration,key,mode,tempo,acousticness,danceability,energy,instrumentalness,liveness,...,anticipation,anger,disgust,fear,joy,positive,sadness,surprise,trust,popularity
0,1921,13.861117,10,1,80.954,0.98200,0.279,0.211,0.878000,0.6650,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1921,3.008883,7,1,60.936,0.73200,0.819,0.341,0.000000,0.1600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1921,8.334367,3,1,110.339,0.96100,0.328,0.166,0.913000,0.1010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1921,3.500000,5,1,100.109,0.96700,0.275,0.309,0.000028,0.3810,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1921,2.778217,3,1,101.665,0.95700,0.418,0.193,0.000002,0.2290,...,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,2020,5.028567,7,1,105.029,0.08460,0.786,0.808,0.000289,0.0822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
170649,2020,2.510900,7,1,137.936,0.20600,0.717,0.753,0.000000,0.1010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
170650,2020,3.521333,4,0,91.688,0.10100,0.634,0.858,0.000009,0.2580,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
170651,2020,5.619117,2,1,75.055,0.00998,0.671,0.623,0.000008,0.6430,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1


## Neural Network

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
X = songs[songs.columns[0:28]].values
Y = songs[songs.columns[28]].values

In [6]:
from sklearn.neural_network import MLPClassifier

In [7]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [8]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'learning_rate':["constant", "invscaling", "adaptive"],
    'solver': ('sgd','lbfgs','adam'),
    'activation': ('logistic','tanh','relu'),
    'hidden_layer_sizes': ((30,),(60,),(80,)),
    'max_iter': (500, 1000)
}
gs = GridSearchCV(estimator = MLPClassifier(), param_grid=parameters,cv=5)
gs.fit(X, Y)
print(gs.best_score_)
print(gs.best_params_)

0.7646388381924444
{'activation': 'relu', 'hidden_layer_sizes': (60,), 'learning_rate': 'adaptive', 'max_iter': 1000, 'solver': 'adam'}


In [9]:
clf = MLPClassifier(solver='adam', hidden_layer_sizes=(60,), max_iter = 1000, 
                    activation='relu',
                    learning_rate='adaptive')
clf.fit(xtrain,ytrain)
predictions = clf.predict(xtest)
actuals = ytest

In [25]:
p_1 = clf.predict(xtrain)
p_2 = clf.predict(xtest)

In [13]:
tp=tn=fp=fn=0
for i in range(len(actuals)):
    a_class=p_class=0
    if int(actuals[i]== 0):
        a_class = 1 
    if int(predictions[i]== 0):
        p_class = 1
    if a_class == 1 and p_class == 1:
        tp +=1
    elif a_class == 1 and p_class == 0:
        fn +=1
    elif a_class == 0 and p_class == 0:
        tn +=1
    elif a_class == 0 and p_class == 1:
        fp +=1
print(tp,tn,fp,fn)
nn_accuracy = ((tp+tn)*100/(tp+tn+fp+fn))
print(nn_accuracy)

8959 16095 1636 7441
73.40540857285166


In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(ytest, p_2,squared=False)

0.5156994417986929

In [27]:
mean_squared_error(ytrain, p_1,squared=False)

0.5137322583528244

In [28]:
from sklearn.metrics import r2_score
r2_score(ytest, p_2)

-0.06540387099347833

In [29]:
r2_score(ytrain, p_1)

-0.056477043994253906